### Format metadata.csv file

In [1]:
import codecs

# remove the BOM tags
with codecs.open('corpus/metadata.csv', 'r', encoding='utf-8-sig') as file:
    content = file.read()

# Write the processed content back to the file
with open('corpus/metadata_cleaned.csv', 'w', encoding='utf-8') as file:
    file.write(content)

### Search keywords in the corpus

* virtual
* reality

In [2]:
import csv
import os
import re

# Step 1: Read metadata.csv
metadata = []
with open("corpus/metadata_cleaned.csv", "r", encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        metadata.append(row)

# Extract header and data
header = metadata[0]
metadata_data = metadata[1:]

# Define phrases to search for
phrases = [
    ("V", r'\b(virtual(?:ity)?)\b'),
    ("R", r'\b(real(?:ity)?)\b'),
]

# Step 2: Traverse each article and each phrase
results = {}

for friendly_phrase, regex_phrase in phrases:
    phrase_results = []
    for article in metadata_data:
        file_path = f"corpus/news/{article[0]}.txt"
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

                # Split the content into sentences
                sentences = [s.strip() for s in content.replace('\n', '. ').split('. ') if s]
                for sentence in sentences:
                    if re.search(regex_phrase, sentence, re.IGNORECASE):
                        # Append the sentence and its metadata to the results list
                        phrase_results.append([sentence] + article)
    results[friendly_phrase] = phrase_results
    

# Step 3: Save the results to separate CSV files with friendly filenames
for friendly_phrase, phrase_results in results.items():
    output_path = f"data/{friendly_phrase}.csv"
    with open(output_path, "w", encoding="utf-8", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Add the header
        csvwriter.writerow(["sentence", "num", "title", "author", "date", "source", "section", "link"])
        # Add the data
        csvwriter.writerows(phrase_results)

    print(f"Results for '{friendly_phrase}' have been saved to {output_path}")

Results for 'V' have been saved to data/V.csv
Results for 'R' have been saved to data/R.csv


### Compute co-word frequencies

* print the most frequent 50 words that co-exist with each keyword

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import pandas as pd

stop_words = set(stopwords.words('english'))
manual_stopwords = ['the','in','i','also','one','two','many','much','may','even','first','last','seem','seems',
                    'could','would','was','were','have','has','well','toward','recently','go','become','able','via',
                    'make','see','kind','use','today','uses','like','new','say','says','said','show','allow','allows',
                    'way','ways','using','used','de','ha','wa','ō','every','with','within','us','every','everyone']
lemmatizer = WordNetLemmatizer()

def compute_word_freq(df):
    """Compute word frequencies for a given DataFrame"""
    words = []
    for sentence in df['sentence']:
        tokens = word_tokenize(sentence)
        # Lemmatize, convert to lowercase, and filter out stopwords and punctuation
        words.extend([lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha() and 
                      lemmatizer.lemmatize(word.lower()) not in stop_words and lemmatizer.lemmatize(word.lower()) not in manual_stopwords])
    return Counter(words).most_common(100)

# List all CSV files in the data/ directory
csv_files = [f for f in os.listdir("data/") if f.endswith('.csv')]

# Dictionary to store common words for each CSV file
common_words_dict = {}

# Iterate over each CSV file, read it into a DataFrame, and compute the common words
for file in csv_files:
    file_path = os.path.join("data/", file)
    df = pd.read_csv(file_path)
    common_words_dict[file] = compute_word_freq(df)

# Print results with spaces between different CSV files
for file, words in common_words_dict.items():
    print(f"Results for {file}:\n")
    for word, count in words:
        print(f"{word}: {count}")
    print("\n" + "-"*50 + "\n")

Results for V.csv:

virtual: 315
reality: 144
art: 69
space: 52
world: 47
artist: 44
work: 40
gallery: 39
exhibition: 36
experience: 30
fair: 29
museum: 24
time: 22
real: 21
augmented: 20
video: 20
technology: 19
online: 19
vr: 16
life: 16
painting: 16
digital: 16
around: 15
physical: 15
visitor: 15
headset: 15
project: 14
people: 14
game: 14
part: 13
installation: 13
year: 12
place: 12
view: 12
think: 12
london: 11
viewer: 11
version: 11
object: 11
screen: 11
director: 10
take: 10
medium: 10
visit: 10
audience: 9
platform: 9
create: 9
york: 9
present: 9
studio: 9
sculpture: 8
future: 8
computer: 8
scene: 8
collector: 8
created: 8
piece: 8
public: 8
explore: 8
met: 8
image: 8
statement: 7
event: 7
live: 7
room: 7
news: 7
still: 7
called: 7
immersive: 7
u: 7
material: 7
give: 7
city: 7
including: 7
wall: 7
app: 7
film: 7
user: 7
study: 7
seeing: 6
interest: 6
artwork: 6
ar: 6
local: 6
global: 6
viewing: 6
never: 6
curator: 6
tour: 6
month: 6
recent: 6
biennale: 6
something: 6
design: 6


### LDA Topic Modeling 

### 1) virtual, virtuality

In [4]:
import gensim
from gensim import corpora
from nltk.stem import SnowballStemmer
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models

V_df = pd.read_csv('data/V.csv')
print('A total of',V_df.shape[0],'corpora are included.')

stemmer = SnowballStemmer("english")

# 1. Text preprocessing
def preprocess(text):
    """Function to preprocess text: tokenize, turn to lowercase, remove stopwords, lemmatize, and stem"""
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in manual_stopwords]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return [token for token in tokens if token.isalpha()]

processed_docs = [preprocess(doc) for doc in V_df['sentence']]

# 2. Construct dictionary and corpus
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# 3. Apply LDA model
num_topics = 5  
lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# 4. Analyse the results
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


A total of 327 corpora are included.
Topic: 0 
Words: 0.049*"virtual" + 0.022*"realiti" + 0.012*"galleri" + 0.008*"art" + 0.006*"exhibit" + 0.006*"peopl" + 0.006*"world" + 0.006*"fair" + 0.006*"game" + 0.006*"artist"
Topic: 1 
Words: 0.048*"virtual" + 0.013*"realiti" + 0.010*"exhibit" + 0.009*"space" + 0.007*"work" + 0.006*"world" + 0.005*"artist" + 0.005*"year" + 0.005*"art" + 0.005*"video"
Topic: 2 
Words: 0.048*"virtual" + 0.034*"realiti" + 0.015*"art" + 0.012*"world" + 0.011*"experi" + 0.008*"artist" + 0.007*"work" + 0.006*"time" + 0.006*"fair" + 0.005*"technolog"
Topic: 3 
Words: 0.041*"virtual" + 0.015*"space" + 0.014*"art" + 0.013*"realiti" + 0.008*"real" + 0.007*"work" + 0.006*"artist" + 0.006*"physic" + 0.006*"exhibit" + 0.005*"view"
Topic: 4 
Words: 0.029*"virtual" + 0.012*"realiti" + 0.011*"artist" + 0.006*"work" + 0.006*"art" + 0.006*"museum" + 0.005*"medium" + 0.005*"view" + 0.005*"monument" + 0.004*"space"


In [5]:
# 5. Use pyLDAvis to visualise
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [6]:
# Get the topic distribution of a document
def get_document_topics(doc_bow):
    return lda_model.get_document_topics(doc_bow)

# Apply the function to each document in the corpus
document_topics = [get_document_topics(doc) for doc in corpus]

# Print topic distribution for each sentence
for i, topics in enumerate(document_topics):
    print(f"Sentence {i+1} topic distribution: {topics}")

Sentence 1 topic distribution: [(0, 0.014449034), (1, 0.014405813), (2, 0.014469642), (3, 0.942322), (4, 0.0143535)]
Sentence 2 topic distribution: [(0, 0.9457622), (1, 0.013459676), (2, 0.013597845), (3, 0.013749471), (4, 0.013430738)]
Sentence 3 topic distribution: [(0, 0.011872652), (1, 0.011869682), (2, 0.011872608), (3, 0.012044691), (4, 0.95234036)]
Sentence 4 topic distribution: [(0, 0.016776765), (1, 0.016797645), (2, 0.9328163), (3, 0.016797729), (4, 0.016811553)]
Sentence 5 topic distribution: [(0, 0.012611676), (1, 0.012627849), (2, 0.9495304), (3, 0.012668902), (4, 0.012561146)]
Sentence 6 topic distribution: [(0, 0.0135644805), (1, 0.013802872), (2, 0.013777419), (3, 0.9452552), (4, 0.013600005)]
Sentence 7 topic distribution: [(3, 0.97284895)]
Sentence 8 topic distribution: [(2, 0.96333975)]
Sentence 9 topic distribution: [(3, 0.96301126)]
Sentence 10 topic distribution: [(0, 0.022458125), (1, 0.909953), (2, 0.022577763), (3, 0.022619346), (4, 0.022391738)]
Sentence 11 to

In [7]:
# View the text content of the corpus corresponding to each Topic
topic_assignments = {}

for i, topics in enumerate(document_topics):
    # Assign the topic with the highest probability to the document
    dominant_topic = sorted(topics, key=lambda x: x[1], reverse=True)[0][0]
    if dominant_topic not in topic_assignments:
        topic_assignments[dominant_topic] = []
    topic_assignments[dominant_topic].append(V_df['sentence'].iloc[i])

# Print sentences for each topic
for topic, sentences in topic_assignments.items():
    print(f"\nTopic {topic} has the following sentences:")
    for sentence in sentences:
        print(f"   {sentence}")


Topic 3 has the following sentences:
   Since diners are unable to see their food in the real world, an animated rendering of each one-bite course appears in the virtual space as it’s served
   “To make virtual spaces into spaces of embodied exploration is a long-held interest of mine,” said Eliasson in a statement to the press about the project, which he has been working on for six years
   Daniel Birnbaum, the artistic director of Acute Art, a company spearheading the augmented reality and virtual reality artworks, told The Art Newspaper that while many artists may have been “a bit hesitant” to enter into the NFT space due to the legitimate environmental concern, that new technologies can also be used to further concepts in art.
   The museum has partnered with HTC VIVE Arts to host a live preview of the exhibition in a virtual reality setting that blends elements of the V&A’s physical museum space with the fantastical world of Wonderland
   Unlike online viewing rooms, with Untitle

### 2) real, reality

In [16]:
R_df = pd.read_csv('data/R.csv')
print('A total of',R_df.shape[0],'corpora are included.')

processed_docs = [preprocess(doc) for doc in R_df['sentence']]
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

num_topics = 5  
lda_model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


A total of 383 corpora are included.
Topic: 0 
Words: 0.025*"realiti" + 0.020*"real" + 0.010*"virtual" + 0.009*"work" + 0.008*"artist" + 0.008*"world" + 0.008*"time" + 0.007*"van" + 0.007*"gogh" + 0.006*"violenc"
Topic: 1 
Words: 0.028*"realiti" + 0.020*"virtual" + 0.014*"experi" + 0.012*"real" + 0.010*"work" + 0.009*"artist" + 0.008*"world" + 0.007*"art" + 0.006*"exhibit" + 0.006*"creat"
Topic: 2 
Words: 0.027*"realiti" + 0.012*"virtual" + 0.011*"real" + 0.010*"life" + 0.007*"game" + 0.006*"work" + 0.005*"augment" + 0.005*"exhibit" + 0.004*"art" + 0.004*"onlin"
Topic: 3 
Words: 0.056*"realiti" + 0.020*"virtual" + 0.019*"art" + 0.017*"augment" + 0.010*"work" + 0.010*"experi" + 0.009*"artist" + 0.007*"ar" + 0.006*"world" + 0.006*"real"
Topic: 4 
Words: 0.034*"realiti" + 0.032*"virtual" + 0.021*"real" + 0.010*"museum" + 0.008*"world" + 0.008*"augment" + 0.007*"physic" + 0.007*"experi" + 0.007*"space" + 0.006*"art"


In [17]:
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

###  3) Corpora

In [14]:
# 1. Text preprocessing
processed_news = []
path = "corpus/news/"

for i in range(1, 176):  # Since there are 175 files
    with open(os.path.join(path, f"{i}.txt"), "r", encoding="utf-8") as file:
        content = file.read()
        processed_content = preprocess(content)
        processed_news.append(processed_content)
        
# 2. Construct dictionary and corpus
dictionary_news = corpora.Dictionary(processed_news)
corpus_news = [dictionary_news.doc2bow(doc) for doc in processed_news]

# 3. Apply LDA model
lda_model_news = gensim.models.LdaMulticore(corpus_news, num_topics=3, id2word=dictionary_news, passes=10)

# 4. Analyse results
for idx, topic in lda_model_news.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.010*"work" + 0.010*"artist" + 0.009*"art" + 0.005*"paint" + 0.004*"exhibit" + 0.004*"world" + 0.003*"experi" + 0.003*"video" + 0.003*"digit" + 0.003*"virtual"
Topic: 1 
Words: 0.010*"work" + 0.009*"art" + 0.008*"artist" + 0.005*"realiti" + 0.005*"virtual" + 0.005*"world" + 0.005*"paint" + 0.005*"exhibit" + 0.004*"experi" + 0.004*"technolog"
Topic: 2 
Words: 0.014*"art" + 0.010*"work" + 0.009*"galleri" + 0.008*"artist" + 0.005*"fair" + 0.004*"virtual" + 0.004*"onlin" + 0.004*"exhibit" + 0.004*"space" + 0.004*"van"


In [15]:
# 5. Use pyLDAvis to visualise
# Normalize topic distributions
topic_term_dists = lda_model_news.get_topics()
topic_term_dists /= topic_term_dists.sum(axis=1)[:, None]

# Visualize
lda_display = pyLDAvis.gensim_models.prepare(topic_model=lda_model_news,
                                             topic_term_dists=topic_term_dists, 
                                             corpus=corpus_news, 
                                             dictionary=dictionary_news, 
                                             sort_topics=False)
pyLDAvis.display(lda_display)

In [12]:
# Get the distribution of topics for each news article
def get_document_topics(doc_bow):
    return lda_model_news.get_document_topics(doc_bow)

# Apply the function to each document in the corpus
document_topics = [get_document_topics(doc) for doc in corpus_news]

# Print topic distribution for each news article
for i, topics in enumerate(document_topics):
    print(f"News {i+1} topic distribution: {topics}")

News 1 topic distribution: [(2, 0.19219), (3, 0.8054542)]
News 2 topic distribution: [(3, 0.9975942)]
News 3 topic distribution: [(1, 0.99658686)]
News 4 topic distribution: [(1, 0.3157619), (2, 0.6820186)]
News 5 topic distribution: [(2, 0.024171317), (3, 0.97381634)]
News 6 topic distribution: [(2, 0.99865377)]
News 7 topic distribution: [(2, 0.9955297)]
News 8 topic distribution: [(2, 0.99709857)]
News 9 topic distribution: [(2, 0.9991941)]
News 10 topic distribution: [(2, 0.9983364)]
News 11 topic distribution: [(3, 0.9956877)]
News 12 topic distribution: [(2, 0.997329)]
News 13 topic distribution: [(1, 0.9960886)]
News 14 topic distribution: [(2, 0.9973544)]
News 15 topic distribution: [(1, 0.9972867)]
News 16 topic distribution: [(2, 0.9981262)]
News 17 topic distribution: [(2, 0.9974793)]
News 18 topic distribution: [(2, 0.99630034)]
News 19 topic distribution: [(2, 0.9982357)]
News 20 topic distribution: [(1, 0.99760276)]
News 21 topic distribution: [(1, 0.99749595)]
News 22 to

In [13]:
# View the news articles corresponding to each Topic
topic_assignments = {}

for i, topics in enumerate(document_topics):
    # Assign the topic with the highest probability to the document
    dominant_topic = sorted(topics, key=lambda x: x[1], reverse=True)[0][0]
    if dominant_topic not in topic_assignments:
        topic_assignments[dominant_topic] = []
    topic_assignments[dominant_topic].append(i+1)  # i+1 is the news number

# Print news articles for each topic
for topic, news_list in topic_assignments.items():
    print(f"Topic {topic} has the following news articles: {news_list}")

Topic 3 has the following news articles: [1, 2, 5, 11, 23, 25, 29, 30, 33, 41, 54, 56, 71, 75, 76, 84, 87, 94, 96, 103, 109, 123, 127, 132, 134, 143, 145, 156, 157, 168, 173]
Topic 1 has the following news articles: [3, 13, 15, 20, 21, 27, 36, 39, 40, 42, 45, 50, 51, 52, 57, 58, 66, 67, 68, 73, 77, 83, 91, 95, 99, 101, 102, 115, 117, 118, 119, 121, 124, 125, 128, 131, 135, 142, 147, 148, 149, 150, 152, 153, 155, 158, 159, 160, 161, 162, 164, 166, 169, 171, 172, 174, 175]
Topic 2 has the following news articles: [4, 6, 7, 8, 9, 10, 12, 14, 16, 17, 18, 19, 22, 24, 26, 28, 31, 32, 34, 35, 37, 43, 47, 48, 49, 53, 55, 59, 60, 61, 63, 64, 65, 69, 72, 74, 78, 79, 80, 81, 82, 85, 86, 88, 89, 90, 92, 93, 97, 98, 100, 105, 107, 108, 110, 111, 112, 113, 114, 116, 120, 122, 126, 129, 130, 133, 136, 138, 139, 141, 144, 146, 151, 154, 167]
Topic 0 has the following news articles: [38, 44, 46, 62, 70, 104, 106, 137, 140, 163, 165, 170]
